<a href="https://colab.research.google.com/github/sergiocostaifes/PPCOMP_DM/blob/main/datasets/04_window_5min_episodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04_window_5min_episodes.ipynb

## Objetivo
Detectar automaticamente **episódios críticos** na série temporal agregada em janelas de **5 minutos**, utilizando limiar estatístico **μ + 2σ**, consolidando intervalos contínuos e gerando métricas por episódio.

## Entradas (do Notebook 03)
- `window_5min_base.parquet`
- `window_5min_series.parquet` (preferencial: série reindexada e contínua)

## Saída (deste Notebook)
- `episodes_detected.parquet`

## Definições
- Série temporal com granularidade oficial de **5 min**
- Métrica principal para detecção: `failures_total` (fallback: `fail_rate`)
- Limiar: `threshold = μ + 2σ`
- Episódio: intervalo contínuo em que `metric >= threshold`

## Métricas por episódio
- `start_ts`, `end_ts`
- `duration_windows` (número de janelas)
- `duration_minutes` (duration_windows * 5)
- `max_value` (intensidade máxima)
- `mean_value` (intensidade média)
- `sum_value` (intensidade total)
- `threshold`, `mu`, `sigma`
- `n_peaks` (número de janelas acima do limiar no episódio)

## Observações de reprodutibilidade
- Bootstrap via Drive + clone/atualização do repositório
- Uso de `src.paths` e `ensure_dirs()`
- Seed global fixa
- Sem dependência de caminhos relativos frágeis

In [2]:
# ============================================================
# 04_window_5min_episodes.ipynb
# Detecção de episódios críticos (μ + 2σ) na série 5-min
# Pipeline PPCOMP_DM (Google Cluster Trace)
# ============================================================

# -----------------------------
# 0) BOOTSTRAP (Colab + Repo)
# -----------------------------
from pathlib import Path
import os
import sys
import subprocess
import importlib
import random
import numpy as np
import pandas as pd

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Mount Google Drive (seguro)
if not Path("/content/drive/MyDrive").exists():
    from google.colab import drive
    drive.mount("/content/drive")
else:
    print("[Bootstrap] Google Drive já montado.")

# Ajuste aqui se o seu repo estiver em outro caminho do Drive
REPO_DIR = Path("/content/drive/MyDrive/Mestrado/PPCOMP_DM")
GITHUB_REPO = "https://github.com/sergiocostaifes/PPCOMP_DM.git"

if not REPO_DIR.exists():
    REPO_DIR.parent.mkdir(parents=True, exist_ok=True)
    print(f"[Bootstrap] Clonando repositório em: {REPO_DIR}")
    subprocess.run(["git", "clone", GITHUB_REPO, str(REPO_DIR)], check=True)
else:
    # Atualiza repo sem quebrar notebook se houver alterações locais
    try:
        print("[Bootstrap] Atualizando repositório (git pull)...")
        subprocess.run(["git", "-C", str(REPO_DIR), "pull"], check=True)
    except Exception as e:
        print("[Bootstrap] Aviso: não foi possível atualizar via git pull:", e)

# Trabalhar sempre a partir do repo (evita path relativo quebrar)
os.chdir(str(REPO_DIR))
print("[Bootstrap] CWD =", os.getcwd())

# Garantir import do pacote src/
repo_str = str(REPO_DIR)
if repo_str not in sys.path:
    sys.path.insert(0, repo_str)
importlib.invalidate_caches()

# Paths padronizados
from src.paths import PROCESSED_PATH, FEATURES_PATH, ensure_dirs
ensure_dirs()

print("PROCESSED_PATH =", PROCESSED_PATH)
print("FEATURES_PATH  =", FEATURES_PATH)

# -----------------------------
# 1) INPUTS (Notebook 03)
# -----------------------------
# Preferimos a série contínua (reindexada) para detecção consistente
inp_series = FEATURES_PATH / "window_5min_series.parquet"
inp_base   = FEATURES_PATH / "window_5min_base.parquet"

if inp_series.exists():
    df = pd.read_parquet(inp_series)
    source_used = "window_5min_series.parquet"
elif inp_base.exists():
    df = pd.read_parquet(inp_base)
    source_used = "window_5min_base.parquet"
else:
    raise FileNotFoundError(
        f"Não encontrei inputs do Notebook 03 em FEATURES_PATH.\n"
        f"Esperado: {inp_series} (preferencial) ou {inp_base}.\n"
        f"Rode o 03_window_5min_base.ipynb primeiro."
    )

print("[Input] Usando:", source_used)
print("[Input] shape:", df.shape)
df.head()

# -----------------------------
# 2) NORMALIZAÇÃO MÍNIMA DE COLUNAS
# -----------------------------
# Esperado: minute_bucket e métricas agregadas
# Métrica principal: failures_total (fallback fail_rate)
# Também aceitamos events_total / fail_rate conforme README
required_time_col = "minute_bucket"
if required_time_col not in df.columns:
    # fallback: timestamp
    if "timestamp" in df.columns:
        df["minute_bucket"] = pd.to_datetime(df["timestamp"], errors="coerce")
    else:
        raise KeyError("Dataset não contém 'minute_bucket' nem 'timestamp' para base temporal.")

df["minute_bucket"] = pd.to_datetime(df["minute_bucket"], errors="coerce")
df = df.dropna(subset=["minute_bucket"]).sort_values("minute_bucket").reset_index(drop=True)

metric_candidates = ["failures_total", "fail_rate"]
metric = None
for c in metric_candidates:
    if c in df.columns:
        metric = c
        break

if metric is None:
    raise KeyError(
        "Não encontrei nenhuma métrica de falhas esperada.\n"
        "Esperado ao menos uma destas colunas: failures_total, fail_rate.\n"
        f"Colunas disponíveis: {list(df.columns)[:50]} ..."
    )

print("[Detecção] Métrica escolhida:", metric)

# Garantir numérico
df[metric] = pd.to_numeric(df[metric], errors="coerce").fillna(0.0)

# -----------------------------
# 3) DETECÇÃO POR LIMIAR (μ + 2σ)
# -----------------------------
mu = float(df[metric].mean())
sigma = float(df[metric].std(ddof=0))  # população (consistência)
threshold = mu + 2.0 * sigma

df["is_critical"] = (df[metric] >= threshold).astype(int)

print(f"[Stats] mu={mu:.6f} sigma={sigma:.6f} threshold={threshold:.6f}")
print("[Stats] critical windows:", int(df["is_critical"].sum()), "de", len(df))

# -----------------------------
# 4) CONSOLIDAÇÃO DE INTERVALOS CONTÍNUOS (EPISÓDIOS)
# -----------------------------
# Episódio = sequência contígua de janelas com is_critical=1
episodes = []
in_episode = False
start_idx = None

for i, flag in enumerate(df["is_critical"].values):
    if flag == 1 and not in_episode:
        in_episode = True
        start_idx = i
    elif flag == 0 and in_episode:
        end_idx = i - 1
        episodes.append((start_idx, end_idx))
        in_episode = False
        start_idx = None

# Se terminou dentro de episódio
if in_episode and start_idx is not None:
    episodes.append((start_idx, len(df) - 1))

print("[Episodes] Detectados:", len(episodes))

# -----------------------------
# 5) MÉTRICAS POR EPISÓDIO
# -----------------------------
WINDOW_MINUTES = 5

rows = []
for ep_id, (s, e) in enumerate(episodes, start=1):
    seg = df.iloc[s:e+1].copy()

    start_ts = seg["minute_bucket"].iloc[0]
    end_ts   = seg["minute_bucket"].iloc[-1]

    duration_windows = int(len(seg))
    duration_minutes = int(duration_windows * WINDOW_MINUTES)

    max_value  = float(seg[metric].max())
    mean_value = float(seg[metric].mean())
    sum_value  = float(seg[metric].sum())

    n_peaks = int(seg["is_critical"].sum())  # aqui será igual ao tamanho do seg, mas mantém semântica

    rows.append({
        "episode_id": ep_id,
        "start_ts": start_ts,
        "end_ts": end_ts,
        "duration_windows": duration_windows,
        "duration_minutes": duration_minutes,
        "metric": metric,
        "max_value": max_value,
        "mean_value": mean_value,
        "sum_value": sum_value,
        "n_peaks": n_peaks,
        "threshold": threshold,
        "mu": mu,
        "sigma": sigma
    })

episodes_df = pd.DataFrame(rows)

episodes_df.head()

# -----------------------------
# 6) SANITY CHECKS
# -----------------------------
if len(episodes_df) > 0:
    # Garantir start <= end
    assert (episodes_df["start_ts"] <= episodes_df["end_ts"]).all()
    # Ordenação
    episodes_df = episodes_df.sort_values(["start_ts", "end_ts"]).reset_index(drop=True)

print("[Episodes] shape:", episodes_df.shape)
episodes_df.describe(include="all")

# -----------------------------
# 7) PERSISTÊNCIA (Saída do Notebook 04)
# -----------------------------
outp = FEATURES_PATH / "episodes_detected.parquet"
episodes_df.to_parquet(outp, index=False)

print("[Output] Salvo:", outp)
print("[Done] Notebook 04 concluído.")

Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '../03-aed/dataset_consolidado.parquet'